 Я реализовал LLM-фгента при помощи LangGraph и протестировал его на задаче оценки релевантности организаций пользовательским запросам. Для запуска нужно вручную загрузить файл с данными data_final_for_dls.jsonl в папку content, а также добавить в секреты блокнота свои API-ключи поисковика tavily и сервиса Vse_GPT.

# Подготовка

In [35]:
! pip -q install ollama langgraph langchain duckduckgo-search tavily-python langchain_community tqdm requests openai beautifulsoup4 selenium chardet requests scikit-learn ddgs graphviz grandalf

In [36]:
# Download Chromium and Chromedriver v114
!wget -q https://storage.googleapis.com/chrome-for-testing-public/114.0.5735.90/linux64/chrome-linux64.zip
!wget -q https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip

# Extract both archives
!unzip -q chrome-linux64.zip
!unzip -q chromedriver_linux64.zip

# Move to proper locations
!mv chrome-linux64 /opt/chrome
!mv chromedriver /opt/chrome/chromedriver
!chmod +x /opt/chrome/chromedriver
!ln -sf /opt/chrome/chromedriver /usr/bin/chromedriver

replace LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [37]:
import ollama
from tqdm import tqdm
import pandas as pd
import langchain
import langchain_community
import langgraph
from duckduckgo_search import DDGS
import time

from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOllama
from duckduckgo_search import DDGS
from bs4 import BeautifulSoup
import requests

from langchain_core.output_parsers import StrOutputParser
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

import chardet
from tavily import TavilyClient
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import os
from google.colab import userdata
import pickle
from IPython.display import clear_output

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Загрузка данных

Здесь требуется загрузать файл с данными data_final_for_dls.jsonl в папку content вручную.


In [4]:
data = pd.read_json(path_or_buf="/content/data_final_for_dls.jsonl", lines=True)

In [5]:
train_data = data[570:]
eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]

# Baseline

In [38]:
Vse_GPT_key = userdata.get('Vse-GPT_key')

llm = ChatOpenAI(
    openai_api_key=Vse_GPT_key,
    openai_api_base="https://api.vsegpt.ru/v1",
    model_name="deepseek/deepseek-chat-0324-alt-fast",  # or deepseek-coder-instruct on site deepseek-reasoner
    temperature=0.0
)

In [8]:
# 0.518900 rub for IFNS request no-sale time
pred_relevancies = []
for index, organization in tqdm(eval_data.iterrows()):

    prompt = f"""You are an agent who evaluates the relevancy of given organization to user request.
    The user request is {organization['Text']}, organization data has name {organization['name'].split(';')[:2]},
    address {organization['address']}, category {organization['normalized_main_rubric_name_ru']},
    information about prices {organization['prices_summarized']} and summarized reviews {organization['reviews_summarized']}.
    You must answer with only one number: 0.0 if organization is irrelevant to user requiest or 1.0 if organization is completely relevant to user requiest.
    Only these two numbers (0.0 or 1.0) are allowed in answer. Your answer must contain only number, no additions allowed."""

    response = llm.invoke([HumanMessage(content=prompt)])

    pred_relevancies.append(float(response.content))

500it [13:58,  1.68s/it]


In [10]:
gt_relevancies = eval_data['relevance'].to_list()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print('Baseline accuracy:', accuracy_score(gt_relevancies, pred_relevancies))
print('Baseline F1 score:', f1_score(gt_relevancies, pred_relevancies))

Baseline accuracy: 0.668
Baseline F1 score: 0.710801393728223


In [ ]:
with open('baseline_eval.pkl', 'wb') as f:
    pickle.dump(pred_relevancies, f)

# Agent

In [39]:
llm = ChatOpenAI(
    openai_api_key=Vse_GPT_key,
    openai_api_base="https://api.vsegpt.ru/v1",
    model_name="deepseek/deepseek-chat-0324-alt-fast",  # or deepseek-coder-instruct on site deepseek-reasoner
    temperature=0.1
)

In [40]:
def read_url_selenium(url: str) -> str:
    '''
    Функция принимает URL (str) и возвращает обработанный текст из html-файла (str) при помощи selenium
    '''
    try:
        options = Options()
        options.binary_location = "/opt/chrome/chrome"
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=options)

        driver.get(url)
        html = driver.page_source
        driver.quit()

        # Encode as bytes
        raw_bytes = html.encode("utf-8", errors="ignore")  # assume utf-8 output, ignore errors

        # Detect encoding from raw bytes
        encoding_guess = chardet.detect(raw_bytes)['encoding']

        # Decode bytes into proper string using detected encoding
        html_text = raw_bytes.decode(encoding_guess or "utf-8", errors="replace")

        # считывание html-файла и извлечение из него текста по тегу div
        soup = BeautifulSoup(html, "html.parser")
        text = " ".join(p.get_text(strip=True) for p in soup.find_all("div"))

        return text[:2000]

    except Exception as e:
        return f"Selenium failed: {e}"

def read_urls(state):
    '''
    Функция принимает словарь state со всей информацией и добавляет в него текст,
    обнаруженный по имеющимся url, сначала используется requests, если он не справляется,
    то selenium
    '''
    texts = []
    state.setdefault("web_texts", [])
    cutoff = len(state["web_texts"])
    for url in state["urls"][cutoff:]:
        try:
            response = requests.get(url, timeout=10)

            # подбор кодировки для корректного считывания текста
            if response.encoding is None or response.encoding.lower() in ['iso-8859-1', 'utf-8', 'ascii']:
                # Some servers default to iso-8859-1 even when it's wrong
                encoding = response.apparent_encoding
                response.encoding = encoding
            else:
                encoding = response.encoding

            # считывание html-файла и извлечение из него текста по тегу div
            soup = BeautifulSoup(response.text, "html.parser")
            text = " ".join(p.get_text(strip=True) for p in soup.find_all("div"))


            if text == '': # проверка, справился ли requests
                text = read_url_selenium(url)
                texts.append(text)

            else:
                texts.append(text[:2000])  # limit for performance

        except Exception as e:
            texts.append(f"Failed to load {url}: {e}")
    state.setdefault("web_texts", []).extend(texts)
    return state

summary_prompt = PromptTemplate.from_template(
'''
Summarize the following content in under 1000 characters, focusing on key details about **{request}**:

**Content:**
{content}

**Rules:**
1. If the content is an error description, respond **only** with: `fail`
2. Exclude meta-commentary (e.g., "Summary is..." or "The content appears...").
3. Keep strictly under 1000 characters.''')

summary_chain = summary_prompt | llm | StrOutputParser()

def summarize_texts(state):
    '''
    Функция принимает словарь state со всей информацией
    и добавляет в него саммари из ранее не саммаризованных веб-текстов.
    Используется модель "deepseek/deepseek-chat-0324-alt-fast" с температурой 0.1

    '''
    state.setdefault('summaries', [])
    cutoff = len(state["summaries"]) # отсечка, чтобы модель работала только с новыми веб-текстами
    summaries = [summary_chain.invoke({"content": text, "request": state['request']}) for text in state["web_texts"][cutoff:]]
    state["summaries"].extend(summaries)
    return state

In [41]:
def decide_search(state):
    '''
    Функция принимает словарь state со всей информацией
    и направляет работу графа на новый поиск, либо завершает ее
    при наличии в state финального ответ модели либо превышении лимита на два поиска.
    '''
    if state["count"] >= 2 or "final_answer" in state.keys():
        state['continue_search'] = "done"
        return state['continue_search']
    else:
        state['continue_search'] = "continue"
        return state['continue_search']

In [42]:
# https://app.tavily.com/home limit check
import random

Tavily_key = userdata.get('Tavily')

client = TavilyClient(api_key=Tavily_key)

def get_tavily_urls(query: str, max_results: int = 3) -> list[str]:
    '''
    Функция получает на вход строку query и число ссылок max_results, которые нужно вернуть.
    Возвращает ссылки в виде списка
    '''
    search_results = client.search(query=query, search_depth="advanced", max_results=max_results)
    urls = [result["url"] for result in search_results["results"]]
    return urls

In [43]:
def search_web_tavily(state):
    '''
    Функция принимает словарь state со всей информацией и запускает функцию get_tavily_urls,
    затем добавляет полученные адреса в state
    '''
    query = state.get("model_request")
    results = get_tavily_urls(query, max_results=2)
    state.setdefault("urls", []).extend(results)
    state["count"] = state.get("count", 0) + 1
    return state

In [44]:
decision_prompt = PromptTemplate.from_template(
    '''
    **Task:** Determine if the organization matches the user’s request.

    **Organization Data:**
    - Name: {name}
    - Address: {address}
    - Category: {category}
    - Prices: {prices}
    - Reviews: {reviews}
    - Additional summaries: {summaries}
    - Count: {count}

    **User’s request: {request}**

    **Rules:**
    1. **If insufficient data**, reply with a **web search query** (<400 chars) to find missing info about given orgainzation.
      - Format: `Query: [your search here]`
    2. **If count == 2 or data is sufficient**, respond **only** with:
      - `0.0` (irrelevant) or `1.0` (fully relevant).
    3. **No extra text** in the final answer.
    ''')

decision_chain = decision_prompt | llm | StrOutputParser()

def decide(state):
    '''
    Функция принимает словарь state со всей информацией и принимает решение,
    достаточно ли информации для ответа.
    Если нет, добавляет в state текст поискового запроса.
    Если да, то добавляет в state окончательный ответ.
    Используется модель "deepseek/deepseek-chat-0324-alt-fast" с температурой 0.1.
    '''
    final = decision_chain.invoke({
        "summaries": "\n\n".join(state["summaries"]),
        "address": state["address"],
        "category": state["category"],
        "prices": state["prices"],
        "reviews": state["reviews"],
        "name": state["name"],
        "request": state["request"],
        'count': state['count']
    })

    if final == '0.0' or final == '1.0':
        state["final_answer"] = final
        return state
    else:
        state["model_request"] = final
        return state

In [45]:
agent_graph = StateGraph(dict)

agent_graph.add_node("search", RunnableLambda(search_web_tavily))
agent_graph.add_node("read_urls", RunnableLambda(read_urls))
agent_graph.add_node("summarize", RunnableLambda(summarize_texts))
agent_graph.add_node("decide", RunnableLambda(decide))
agent_graph.add_node("decide_search", RunnableLambda(decide_search))

# Define flow
agent_graph.set_entry_point("decide")

agent_graph.add_conditional_edges("decide", decide_search, {
    "continue": "search",  # loop again
    "done": END           # exit the graph
})

agent_graph.add_edge("search", "read_urls")
agent_graph.add_edge("read_urls", "summarize")
agent_graph.add_edge("summarize", "decide")

# Compile graph
agent_graph_executor = agent_graph.compile()

Схема графа:
1. Информация поступает в решающую сеть decide
2. Если decide считает, что имеющейся информации недостаточно, то она отвечает текстом поискового запроса и направляет его в модуль search
3. Search отправляет запрос черех tavily и возвращает первые две ссылки из поиска
4. Текст по этим ссылкам затем считывает функция read_urls. Она работает через request, а если request не считывает текст, то подключает selenium
5. Считанный текст отправляется в модуль summarize, где модель пишет короткие саммари на основе этого текста
6. Саммари всесте со всей остальной информацией возвращаются в decide, которая может либо еще один раз совершить поиск (возможно максимум 2 поиска для одного объекта), либо дать финальный ответ: 0.0 либо 1.0

In [24]:
svg = agent_graph_executor.get_graph().print_ascii()
display(svg)  # in Jupyter

                     +-----------+                 
                     | __start__ |                 
                     +-----------+                 
                           *                       
                           *                       
                           *                       
                      +--------+                   
                      | decide |*                  
                   ...+--------+ ****              
               ....        .         ****          
           ....           .              ***       
         ..               .                 ****   
+---------+          +--------+                 ** 
| __end__ |          | search |                  * 
+---------+          +--------+                  * 
                          *                      * 
                          *                      * 
                          *                      * 
                    +-----------+                * 
            

None

In [ ]:
# 1 hour, ~700 roubles
# 420 tavily tokens
from IPython.display import clear_output

agent_predictions = []
for index, organization in tqdm(eval_data.iterrows()): # eval_data
    state = {"model_request": '',
             "request": organization['Text'],
             'name': organization['name'].split(';')[:2],
             'address': organization['address'],
             'category': organization['normalized_main_rubric_name_ru'],
             'prices': organization['prices_summarized'],
             'reviews': organization['reviews_summarized'],
             'summaries': [],
             'count': 0}

    result = agent_graph_executor.invoke(state)
    agent_predictions.append(result)
    clear_output(wait=True)
    '''
    print(state)
    print("Final Answer:\n", result["final_answer"])
    '''

500it [1:05:48,  7.90s/it]


In [ ]:
gt_relevancies = eval_data['relevance'].to_list()

In [ ]:
agent_relevancies = []
for i in agent_predictions:
    agent_relevancies.append(float(i['final_answer']))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print('Agent accuracy:', accuracy_score(gt_relevancies, agent_relevancies))
print('Agent F1 score:', f1_score(gt_relevancies, agent_relevancies))

Agent accuracy: 0.66
Agent F1 score: 0.6816479400749064


In [ ]:
with open('agent_predictions_full_1.pkl', 'wb') as f:
    pickle.dump(agent_predictions, f)